Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Pull data from web and store inside df dataframe
web_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
store = pd.read_html(web_link)
df = store[0]
df.columns = ['PostalCode','Borough','Neighbourhood']

In [5]:
#Remove rows where Borough is not assigned
df = df[df['Borough'] != 'Not assigned']
df.reset_index(drop=True, inplace=True)

In [7]:
#Group by PostalCode
df = df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].agg(lambda x: ', '.join(x))
df = pd.DataFrame(df)
df.reset_index(inplace=True)
df.shape

In [16]:
#Read coordinates and store in dataframe
coords = pd.read_csv('Geospatial_Coordinates.csv')

In [21]:
#Merge Original and coordinate dataframe
df = df.join(coords,how='left',lsuffix='PostalCode',rsuffix='Postal Code')
df.drop('Postal Code', axis=1, inplace=True)
df.head()

In [33]:
#Count number of neighbourhoods in each postalcode
df['Number'] = df['Neighbourhood'].apply(lambda x: len(x))

In [35]:
from sklearn.cluster import KMeans

In [44]:
#Cluster by number of neighbourhood
df['Number2'] = df['Number']
k_means = KMeans(init='k-means++', n_clusters=3, n_init=10)
k_means.fit(df[['Number2','Number']])
df['Label'] = k_means.labels_

In [45]:
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Number,Number2,Label
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,14,14,0
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,38,38,1
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,33,33,1
3,M1G,Scarborough,Woburn,43.770992,-79.216917,6,6,0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,9,9,0


In [57]:
import folium

In [64]:
#Show on map
visual = folium.Map(location=[43.6532, -79.3832], zoom_start=15)
color_list = ['red', 'yellow', 'blue']
for lat, lng, label in zip(df.Latitude, df.Longitude, df.Label):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        fill=True,
        fill_color=color_list[label],
        color=color_list[label]
    ).add_to(visual)

In [65]:
visual